# K-12 database

#### Load python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_grid as altgrid
import glob
from pathlib import Path
import numpy as np

In [3]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

In [10]:
# https://www.chds.us/ssdb/data-map/

In [10]:
incidents = pd.read_excel("data/raw/SSDB_Raw_Data.xlsx", sheet_name="INCIDENT")

In [13]:
incidents.columns = (
    incidents.columns.str.replace(" ", "_", regex=False)
    .str.replace("#", "no", regex=False)
    .str.lower()
)

In [19]:
incidents.head()

,incident_id,sources,number_news,media_attention,reliability,date,quarter,school,city,state,school_level,location,location_type,during_school,time_period,first_shot,summary,narrative,situation,targets,accomplice,hostages,barricade,officer_involved,bullied,domestic_violence,gang_related,preplanned,shots_fired,active_shooter_fbi
0,20200727AZCAO,https://www.kgun9.com/news/education/man-dies-...,3,Local,4,2020-07-27,Summer,Canyon del Oro High School,Oro Valley,AZ,High,Beside Building,Outside on School Property,No,Not a School Day,16:00:00,Adult male construction work fatal shot self i...,40 year-old male fatally shot himself (acciden...,Accidental,Neither,No,No,No,No,No,No,No,No,NaN,No
1,20200721SCBRO,https://abcnews4.com/news/crime-news/person-of...,2,Local,3,2020-07-21,Summer,Brookdale Elementary School,Orangeburg,SC,Elementary,Basketball Court,Outside on School Property,No,Night,9:00 PM,Adult male shot on school basketball court,28 year-old male victim was shot on the school...,Escalation of Dispute,NaN,No,No,No,No,No,No,NaN,No,NaN,No
2,20200714MIWID,https://www.detroitnews.com/story/news/local/m...,1,Local,3,2020-07-14,Summer,Willow Woods Elementary School,Detroit,MI,Elementary,Parking Lot,Outside on School Property,No,Not a School Day,NaN,Shots fired into the air during fight between ...,Shots fired into the air between two men who p...,Escalation of Dispute,Neither,No,No,No,No,No,No,No,No,NaN,No
3,20200704INLAM,https://www.abc57.com/news/murder-suspect-char...,2,Local,4,2020-07-04,Summer,Lake Hills Elementary School,Michigan City,IN,Elementary,Parking Lot,Outside on School Property,No,Night,NaN,Adult male fired shot during exchange of stole...,22 year-old male was involved in a prior assua...,Illegal Activity,Victims Targeted,No,No,No,No,No,No,No,No,NaN,No
4,20200701ILFRP,https://www.dailyherald.com/news/20200714/pros...,1,Local,3,2020-07-01,Summer,Frost Elementary School,Prospect Heights,IL,Elementary,Parking Lot,Outside on School Property,No,Not a School Day,4:35 PM,Adult male fired shot from vehicle at 4 people,25 year-old male fired shots from a vehicle at...,Drive-by Shooting,NaN,Yes,No,No,No,No,No,Yes,No,NaN,No


In [16]:
shooters = pd.read_excel("data/raw/SSDB_Raw_Data.xlsx", sheet_name="SHOOTER")

In [17]:
shooters.columns = (
    shooters.columns.str.replace(" ", "_", regex=False)
    .str.replace("#", "no", regex=False)
    .str.lower()
)

In [20]:
src = pd.merge(incidents, shooters, left_on="incident_id", right_on="incidentid")

In [31]:
src["age"] = src["age"].fillna(np.nan)

In [32]:
len(src[src["active_shooter_fbi"] == "Yes"])

201

In [33]:
active = src[src["active_shooter_fbi"] == "Yes"]

In [37]:
len(active[(active["age"] == "19") | (active["age"] == "18") | (active["age"] == "20")])

27

In [38]:
len(active)

In [42]:
active.gender.value_counts(normalize=True)

Male           0.964103
Female         0.030769
Transgender    0.005128
Name: gender, dtype: float64

In [44]:
len(
    active[
        (active["age"] == "19")
        | (active["age"] == "18")
        | (active["age"] == "20") & (active["summary"].str.contains("assult rifle"))
    ]
)

23